# Target Creation

#### An "adopted user" is a user who has logged into the product on three separate days in at least one seven-day period. I created a function which identified an adopted user

In [4]:
import pandas as pd
import numpy as np

In [5]:
user_engagement = pd.read_csv("takehome_user_engagement-intern.csv",sep=",",error_bad_lines=False)
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'], format="%Y-%m-%d")
user_engagement['time_stamp'] = user_engagement['time_stamp'].dt.date
users = pd.read_csv("takehome_users-intern.csv",sep=",",encoding="ISO-8859-1")

In [6]:
users.shape

(12000, 11)

In [7]:
def adopted_user(user_engagement,users):
    u =  users['object_id'].unique()
    df = pd.DataFrame(columns=["user_id", "target"])
    
    for i in range(1,len(u)+1):
        a = user_engagement[(user_engagement['user_id'] == i)]
        if a.empty == True:
            b = pd.DataFrame(columns=["user_id", "target"], data=[[i,0]])
            df = df.append(b)
            continue
        else:
            x=(a.groupby('user_id')['visited'].agg(['sum']) < 3).reset_index()
            if x['sum'].bool() == True:
                b = pd.DataFrame(columns=["user_id", "target"], data=[[i,0]])
                df = df.append(b)
            else:
                for j in range(0,a.shape[0]-2):
                    
                    if (a.iloc[j+2,0] - a.iloc[j,0]).days < 8:
                        b = pd.DataFrame(columns=["user_id", "target"], data=[[i,1]])
                        df = df.append(b)
                        if b.get_value(0,'target') == 1:
                            break
                        else:
                            continue
                    else:
                        b = pd.DataFrame(columns=["user_id", "target"], data=[[i,0]])
                        df = df.append(b)
    df = pd.DataFrame((df.groupby('user_id')['target'].agg(['max'])).reset_index()).rename(columns={'max':'target'})
    return(df)

In [8]:
target = adopted_user(user_engagement,users)

C:\Users\ppital499\AppData\Local\Continuum\anaconda3.6\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [9]:
target['user_id'].count()

12000

In [10]:
target['target'].sum()

1656